In [1]:
import telepot
from telepot.loop import MessageLoop
import pandas as pd
import pandas as pd
import numpy as np
import pickle
import threading
import glob
import re
import traceback

stock_predictor_bot = telepot.Bot('359528427:AAHKD8rIiFpo0ufZTDvLRGQ9c6gTfFFfYq4')

In [2]:
import logging
import os
import sys
# logger = logging.getLogger()
# logger.setLevel(logging.INFO)
# from pyspark.sql import Row
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")

# Add the py4j to the path.
# You may need to change the version number to match your install
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.6-src.zip'))

# Initialize PySpark to predefine the SparkContext variable 'sc'
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.0
      /_/

Using Python version 3.5.3 (default, Nov 23 2017 11:34:05)
SparkSession available as 'spark'.


In [3]:
import pyspark

from pyspark.ml import PipelineModel
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressionModel
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

days = 29
gbtModel_30_features = PipelineModel.load('../gbtModel_30_features')

In [68]:
def convert_float( x):
    try:
        x['Volume'] = float(x['Volume'])
        x['Close'] = float(x['Close'])
        x['High'] = float(x['High'])
        x['Low'] = float(x['Low'])
        x['Open'] = float(x['Open'])
        return x
    except:
        x['Volume'] = float(x['Volume'])
        x['Close'] = float(x['Close'])
        x['High'] = float(x['Close'])
        x['Low'] = float(x['Close'])
        x['Open'] = float(x['Close'])
        return x
    
def feature_ext(temp=None):
    temp['close_close_gap'] = (temp['Close'].shift(-1) - temp['Close']) / temp['Close']
    temp['close_open_gap'] = (temp['Open'].shift(-1) - temp['Close']) / temp['Close']
    temp['close_high_gap'] = (temp['High'].shift(-1) - temp['Close']) / temp['Close']
    temp['close_low_gap'] = (temp['Low'].shift(-1) - temp['Close']) / temp['Close']
    temp.drop(labels=["High","Low","Open","Close"], axis=1,inplace=True)
    return temp

def get_features(daily_price_data, code, date_list, days_to_be_stacked) :
    features = {}
    print("date list " + str(date_list))
    for date in date_list :
        try :
            selected_index = daily_price_data.index[daily_price_data['Date'] == date].tolist()

            if len(selected_index) == 0 :
                print("date " + date + "is not found")
                continue

            selected_index = selected_index[0]
            df = daily_price_data[selected_index + 1 : selected_index + days_to_be_stacked + 2]
            df = df.reset_index().drop('index',axis=1)
            df = df.drop(labels=['Date'], axis=1)
            df = df.apply(lambda x:convert_float(x),axis=1)
            feature_ext(df)
            df = df[0:days_to_be_stacked]
            df = df.stack().to_frame().T
            df.columns = ['{}_{}'.format(*c) for c in df.columns]
#             features.append(df.values[0].tolist())
            features[date] = df.values[0].tolist()
        except Exception as e:
            traceback.print_exc()
            print("exception when get feature " + str(e) + " for date " + date)
    
    return features

def handle_message(msg):
    print('updated 10')
    content_type, chat_type, chat_id = telepot.glance(msg)

    message = ""
    username = msg["from"]["first_name"]
    if content_type == 'text':
        message = msg["text"]
        if 'predict' in message :
            message = message.split('predict')[1].lstrip()
            data = message.split('\n')
            stock_code = data[0].lstrip()
            date_list = data[1:]
            stock_predictor_bot.sendMessage(chat_id, 'prediction starting.. ')
            predict_stock(chat_id, stock_code, date_list)

def predict_stock(chat_id, stock_code, date_list):
    daily_price_data = []
    try :
        daily_price_data = pd.read_csv('../until31_2017/' + stock_code.lstrip() + '.csv')
    except Exception as e:
        stock_predictor_bot.sendMessage(chat_id, 'invalid stock name ')
        print('invalid stock name ' + str(e))
        return
    
    features = []
    try :
        stock_predictor_bot.sendMessage(chat_id, 'gathering features for stock ' + stock_code)
        features = get_features(daily_price_data, stock_code, date_list, days)
        stock_predictor_bot.sendMessage(chat_id, 'features generated for stock ' + stock_code)
    except Exception as e:
        stock_predictor_bot.sendMessage(chat_id, 'failed to generate features')
        print('failed to generate features due to ' + str(e))
        return
    
    try :
        stock_predictor_bot.sendMessage(chat_id, 'predicting for stock ' + stock_code)
        df = sc.parallelize([{'features':features[key], 'date':key} for key in features.keys()]).toDF()
        list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
        final_data = df.select(df['date'], list_to_vector_udf(df["features"]).alias("features"))

        predictions = gbtModel_30_features.transform(final_data)
        predictions = predictions.select("prediction", "date")
        predictions = predictions.collect()
        result = 'prediction result :\n'
        for p in predictions :
            result = result + 'date ' + p['date'] + ' ' + str(round(p['prediction'], 4)) + '\n'
        stock_predictor_bot.sendMessage(chat_id, result)
    except Exception as e :
        traceback.print_exc()
        print("exception when predict stock " + str(e))
    
def start():
    MessageLoop(stock_predictor_bot, handle_message).run_as_thread()
    print ('Stock Predictor Bot Listening ...')

In [ ]:
start()

import time
while 1:
    time.sleep(100)

Stock Predictor Bot Listening ...


Exception in thread Thread-10:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.5/dist-packages/telepot/loop.py", line 148, in run_forever
    self._bot.scheduler.run_as_thread()
  File "/usr/local/lib/python3.5/dist-packages/telepot/__init__.py", line 404, in run_as_thread
    self.daemon = True
  File "/usr/lib/python3.5/threading.py", line 1139, in daemon
    raise RuntimeError("cannot set daemon status of active thread")
RuntimeError: cannot set daemon status of active thread



updated 10
date list ['8-Dec-17', '1-Nov-17', '3-Nov-17']


/home/tryan/spark_tensor/spark-2.3.0-bin-hadoop2.7/python/pyspark/sql/session.py:360: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "
